In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as mpl
from matplotlib import rc
# rc('text',usetex=True)
rc('font',**{'family':'serif','serif':['Times New Roman'],'size':8})
rc('mathtext',fontset='cm')
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}
import matplotlib.gridspec as gridspec

In [3]:
import numpy as np
import random as rn

In [4]:
n = 40
Xs = np.random.randn(150,10)
rs = Xs.sum(axis=1)
rs /= rs.std()

In [17]:
Xs1 = Xs[rs>=0]; r1 = rs[rs>=0]
Xs2 = Xs[rs<0]; r2 = rs[rs<0]

In [6]:
fig,ax = plt.subplots(1,2,figsize=(8,4))
ax1,ax2 = ax
axcb = fig.add_axes([0.123,0.05,0.355,0.05])
ax1.set_aspect('equal')
ax2.set_aspect('equal')

c = ax1.scatter(Xs[:,0],Xs[:,1],c=rs,cmap='seismic')
cb = fig.colorbar(c,cax=axcb,use_gridspec=True,orientation='horizontal')
cb.set_label('$r$')

c1 = ax2.scatter(Xs1[:,0],Xs1[:,1],marker='+',label='$q(x) = \\alpha$')
c2 = ax2.scatter(Xs2[:,0],Xs2[:,1],marker='_',alpha=0.8,label='$q(x) = -\\alpha$')
ax2.scatter(Xs1[0,0],Xs1[0,1],c='w',alpha=0,label='$q(x) = 0$ ailleurs')
ax2.legend(ncol=3,labelspacing=0,mode='expand',bbox_to_anchor=(0, -1.05, 1, 1),frameon=False)

for ax in [ax1,ax2]:
    ax.axis(ymin=-3.2,ymax=3.2,xmin=-3.2,xmax=3.2)
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.annotate('$X_1$',xy=(0.2,2.8))
    ax.annotate('$X_2$',xy=(2.5,0.2))

ax1.set_title('a) Observations du marché')
ax2.set_title('b) Fonction de décision "dictionnaire"')    

# plt.savefig('fig/demo_1.pdf',bbox_inches='tight',pad_inches=0)
#


In [321]:
import cvxpy as cvx
from scipy.spatial.distance import pdist,cdist,squareform

In [384]:
X = Xs[:,:3]
X[:,2] = 1

In [395]:
sigma = 0.1

In [396]:
K = np.exp(-squareform(pdist(X))**2/sigma)
K.shape

(150, 150)

In [397]:
def solveg(sigma,l,r):
    # Only valid for exponential utility u(r) = -exp(-r)
    K = np.exp(-squareform(pdist(Xs))**2/sigma)
    n,_ = K.shape
    a = cvx.Variable(n)
    ra = cvx.mul_elemwise(r,a)
    obj = -cvx.sum_entries(cvx.kl_div(a,1)) - l*cvx.quad_form(ra,K)
    prob = cvx.Problem(cvx.Maximize(obj))
    prob.solve(solver=cvx.SCS)
    a = a.value.A1
    return a

In [398]:
def phi(sigma,X_s):
    phi = np.exp(-cdist(X,X_s)**2/sigma).T
    return phi

In [417]:
a = solveg(1,0.01,rs)

In [418]:
delta = 150
xx1 = np.linspace(min(X[:,0]),max(X[:,0]),delta)
xx2 = np.linspace(min(X[:,1]),max(X[:,1]),delta)
X_s = np.array(np.meshgrid(xx1,xx2,[1])).T[0].reshape((delta*delta,3))

In [419]:
Z = np.inner(phi(sigma,X_s),a*rs).reshape((delta,delta))

In [420]:
plt.scatter(X[:,0],X[:,1],c=rs,cmap='seismic');
plt.contour(xx1,xx2,Z,10)

In [7]:
import cvxpy as cvx
from scipy.spatial.distance import pdist,cdist,squareform

In [8]:
X = Xs[:,:3]
X[:,2] = 1

In [9]:
sigma = 0.1

In [10]:
K = np.exp(-squareform(pdist(X))**2/sigma)
K.shape

(150, 150)

In [11]:
def solveg(sigma,l,r):
    # Only valid for exponential utility u(r) = -exp(-r)
    K = np.exp(-squareform(pdist(Xs))**2/sigma)
    n,_ = K.shape
    a = cvx.Variable(n)
    ra = cvx.mul_elemwise(r,a)
    obj = -cvx.sum_entries(cvx.kl_div(a,1)) - l*cvx.quad_form(ra,K)
    prob = cvx.Problem(cvx.Maximize(obj))
    prob.solve(solver=cvx.SCS)
    a = a.value.A1
    return a

In [12]:
def phi(sigma,X_s):
    phi = np.exp(-cdist(X,X_s)**2/sigma).T
    return phi

In [13]:
a = solveg(1,0.01,rs)

In [52]:
delta = 150
xx1 = np.linspace(min(X[:,0]),max(X[:,0]),delta)
xx2 = np.linspace(min(X[:,1]),max(X[:,1]),delta)
X_s = np.array(np.meshgrid(xx1,xx2,[1])).T[0].reshape((delta*delta,3))

In [15]:
Z = np.inner(phi(sigma,X_s),a*rs).reshape((delta,delta))

In [16]:
plt.scatter(X[:,0],X[:,1],c=rs,cmap='seismic');
plt.contour(xx1,xx2,Z,10)

In [70]:
from helper import solve

In [90]:
n = 100
Xs = np.random.randn(150,3)
Xs[:,1] *= 10
rs = Xs.sum(axis=1)
rs /= rs.std()
Xs[:,2] = 1

In [91]:
Xs1 = Xs[rs>=0]; r1 = rs[rs>=0]
Xs2 = Xs[rs<0]; r2 = rs[rs<0]

In [103]:
q = cvx.Variable(3)
obj = cvx.Minimize(cvx.sum_squares(rs - Xs*q) + 10*cvx.norm(q)**2)
prob = cvx.Problem(obj)
prob.solve()
q_ls = q.value.A1

In [104]:
q_ls

array([ 0.10128766,  0.10618704,  0.00913582])

In [105]:
delta = 150
xx1 = np.linspace(Xs[:,0].min(),Xs[:,0].max(),delta)
xx2 = np.linspace(Xs[:,1].min(),Xs[:,1].max(),delta)
xx,yy = np.meshgrid(xx1,xx2)
z_ls = q_ls[0]*xx + q_ls[1]*yy + q_ls[2]


In [106]:
plt.scatter(Xs[:,0],Xs[:,1],c=rs,cmap='seismic')
cont = plt.contour(xx,yy,z_ls)
plt.clabel(cont)

<a list of 7 text.Text objects>

In [82]:
q_a = (Xs*rs[:,None]).mean(axis=0)
q_a

array([ 0.44193285,  0.62888695, -0.09663898])

In [98]:
z_a = q_a[0]*xx + q_a[1]*yy + q_a[2]
plt.scatter(Xs[:,0],Xs[:,1],c=rs,cmap='seismic')
cont = plt.contour(xx,yy,z_a)
plt.clabel(cont);


In [76]:
Xs*rs

ValueError: operands could not be broadcast together with shapes (150,3) (150,) 